# Air quality : Transitive frontier learnig Example

SyntaxError: invalid syntax (3570227345.py, line 1)

Below there is an example of Transitive Frontier Learning implementation on the Air quality dataset.
First we showed how the dataset was processed then how the analysis was carryed on.

In [ ]:
import pandas as pd
import datetime as dt
from math import log2  
import pydotplus as ptp 

# helper_classes 
import import_ipynb
import helper_general as my
from ConstraintDefiner import ConstraintDefiner as Lark_CD


## Loading + preprocessing Dataset

In [ ]:
# Take the correct rows and columns
airQuality = pd.read_csv("./AirQualityUCI.csv", sep=';').iloc[0:9356,:-2] 
display(airQuality.head(3))

# Replacement the "," with "." in the scientific notations 
airQuality = airQuality.astype(str).applymap(lambda val: val.replace(',','.'))

# Replacement the "(" and ")" with "_" in the columns' name to prevent grammar's errors
origNames = airQuality.columns.copy()
airQuality.columns = [ colName.replace("(","_").replace(')','').replace('.','_') for colName in airQuality.columns.to_list()]

# Add weekend
airQuality['Date'] = (airQuality['Date'].map(lambda x: dt.datetime.strptime(x,'%d/%m/%Y')).dt.dayofweek >= 5).astype(int)
airQuality = airQuality.rename(columns={'Date':'Weekend'})
display(airQuality.head(3))

## Mapping values, Encoder and Decoder
Mapping

In [ ]:
# Values mapping into integer values
encod , decod = {},{}
for col in airQuality.columns.to_list():
    
    if col == "Time":
        dec = airQuality[col].drop_duplicates().sort_values().reset_index(drop=True)
    elif col == "Weekend":
        dec = airQuality[col].drop_duplicates().sort_values().reset_index(drop=True)
    else :
        dec = airQuality[col].astype(float).drop_duplicates().sort_values() #float-> -200 == -200.0 
        dec = dec.iloc[1:].reset_index(drop=True) # remove the -200.0 (None) => mapping only between not None values    
    enc = dec.reset_index().set_index(col)['index']
    
    encod |= {col: dict(enc)| {-200.0: -1} } # In orther to have only integer values
    decod |= {col: dict(dec)| {-1 : -200.0}}
#my.myDisplay([encod['CO_GT'], decod['CO_GT']], axis=1)

Encoding Dataset

In [ ]:
df_AQ = pd.DataFrame() 
for col in airQuality.columns:
    if col == "Time":
         df_AQ[col] = airQuality[col].map(lambda val: encod[col][val])
    else:
        df_AQ[col] = airQuality[col].map(lambda val: encod[col][float(val)]) # cast in float -> '2' == '2.0' == 2.0
df_AQ

Decoding Dataset

In [2]:
df_AQ_2 = pd.DataFrame() 
for col in airQuality.columns:
         df_AQ_2[col] = df_AQ[col].map(lambda val: decod[col][val])
df_AQ_2

NameError: name 'pd' is not defined

## Decode Air Quality Rule

In [ ]:
def AQ_decodeRule(decod, df_X, df_Y):
    def colDec(col, name):
        return col.map(lambda v: decod[name][v])  
    
    X = df_X.T.apply(lambda col: colDec(col, col.name)).T
    Y = df_Y.T.apply(lambda col: colDec(col, col.name)).T
    
    #--------------------------
    # Pretty rule
    X_pretty = X.index+'['+ X.L.astype(str)+', '+X.U.astype(str)+']'
    Y_pretty = Y.index+'['+ Y.L.astype(str)+', '+Y.U.astype(str)+']'
    s = f"{', '.join(X_pretty.tolist())} --> {', '.join(Y_pretty.tolist())}"
    
    return X,Y,s

# dX,dY,ds = AQ_decodeRule(decod, X,Y) 
# my.myDisplay([dX,dY], axis=1)
# ds

## Support and Confidence

In [ ]:
def supp_conf(df_Coded, df_X, df_Y, tempShow=False):
    df_temp = pd.DataFrame()
    for col in df_X.index:
        df_temp[col] = (df_X['L'][col] <= df_Coded[col]) & (df_Coded[col] <= df_X['U'][col])
    
    for col in df_Y.index:
        df_temp[col] = (df_Y['L'][col] <= df_Coded[col]) & (df_AQ[col] <= df_Y['U'][col])
    
    #-----------------------------------------
    # Calcolo freq X e XY
    freqX = df_temp[ df_temp[df_X.index].T.sum() == len(df_X)] # freq of X 
    freqXY = freqX[freqX[df_Y.index].T.sum() == len(df_Y)]     # freq of XY
    
    if tempShow : my.myDisplay([df_temp,freqX,freqXY],names=['Evaluation:','X:','XY:'], axis=1)
    return {'supp':len(freqXY)/len(df_Coded) , 'conf':0 if len(freqX)==0 else len(freqXY)/len(freqX)}

# X,Y,s = c.decodeRule(res)
# suppConf = supp_conf(df_AQ, X, Y, True)
# suppConf

## Transitive frontier learnig analysis

In [ ]:
# after preprocessing
df_AQ.head(3)

Select the attributes to include in the analysed 

In [ ]:
df_var = df_AQ.max().to_frame().reset_index()
df_var['index']

##  Create an instance of ConstraintDefiner (constrDef)

This class was designed to create an interface between the user and the SAT-like core. 
It takes the variables and their maximum value to generate constraints which will guide the SAT-like search.

Each variable was represented as an interval described whit a lower bound and an upper bound. Moreover, each interval value was used their their binary rappresetations. 

The constraints implemented are: 

    init_constr(): Describes the rule structure and the intervals' rappresentation.
    
    NotSupported(res): Describes the constraint we want to apply if we find out a supported rule.
    
    NotConfident(res): Describes the constraint we want to apply if we find out a not confident rule.
    
    Confident(res):    Describes the constraint we want to apply if we find out a confident rule
    
    reject_solut(res): Describes the constraint we want to apply if we no longer want a rule.
    
Each implemented constraint return a string that should be inserted into a ConstraintDefiner's instance. 
We can insert a new constraint using the following method: .addConstr(constraint)

Before starting we need to create a ConstraintDefiner's instance and insert rule structure's constraint. We can also set a seed for data reproducibility.

In [ ]:
c = Lark_CD(df_var['index'], df_var[0]) 
c.addConstr( c.init_constr() )  
c.set_seed(2981050372)

### Heuristic implemented
We can apply some heuristics to speed up the search by enabling the following method:

###### set_resetTree(False): 
The ConstrDef instance keeps track of the values previously tested in previous searches, so as not to repeat paths already visited. if seleted to True, it will behave like a normal sat-solver

###### set_priority(on_bits=opt)
This heuristic priority set a priority between attributes. In this case, each attribute has the same probability of being assigned a value. 

The 'on_bits' set a priority over the value's bits, so we can consider the most significant bit first. There are three options implemented:

    - none: There is no priority setted.
    - 'Simple': In each interval values, will be evaluate the most significant bit first. 
    - 'Complete' First, will be evaluate the most significant bit among all the interval values, then the others.



###### set_saturate_freeVars(opt):


This heuristic set a priority

###### set_pathExplorer(opt):
This heuristic represent all the possible set a different method to explore the possible paths which will lead to a solutions.

There options implemented are:
    
    random: Eac
    


 we can set each heulistic as shown below:

In [ ]:
c.set_seed(2981050372)  # we can set a seed to reproducibility
c.set_pathExplorer('soft_max') #c.set_childrenChoice('min_power') #'random' or 'min_avgLenPath' or 'min_power' (soft_max)
c.set_priority(on_bits='Complete') # none, 'Simple', 'Complete'
c.set_saturate_freeVars(saturate='effective') # None, 'all', 'effective'
c.set_resetTree(False) 

#### Here there is an instance of analysis setup:

In [ ]:
# Decide support and confidence and set the durations
supp = 0.05
conf = 0.8
duration = '10m' # test durations (s:seconds, m:minutes, h:hours)

In [ ]:
i, allBool = 0, False
sol, state_Sol, data_Solut= [], [], []

#--------------------
duration = pd.Timedelta(duration)
startTime = dt.datetime.now() 

#---------------------------
while (dt.datetime.now() - startTime ) < duration:  
    start_run = dt.datetime.now()
    nChar = sum([len(constr) for constr in c.constr])
    print(f"Run:{i+1}, #char:{nChar}, Started:{start_run.strftime('%m-%d %H:%M:%S')}, ", end='')
    print("")
    #-----------------------
    # SAT
    res = c.Sat_Solver(test='speed') # only for test, else:  c.Sat_Solver()
    end_run = dt.datetime.now()
    
    if res is None: break    # noSoluzione
    if len(res)==0:  
        allBool=True    # anly true and false -> ris={}
        break
       
    #-----------------------
    # Supp + Conf
    X,Y,s = c.decodeRule(res) # X -> antecedent, Y-> consequent, L->lower bount, U->upper bound
    suppConf = supp_conf(df_AQ, X, Y)
    
    sol = {'solut':s } | suppConf
    if suppConf['supp'] < supp:
        c.addConstr( c.NotSupported(res) )
        sol |= {'state':'Not supported'}
    
    elif suppConf['conf'] < conf:
        c.addConstr( c.NotConfident(res) )
        sol |= {'state':'Not confident'}
    
    else:
        c.addConstr( c.Confident(res) )
        sol |= {'state':'Confident'}
    
    c.addConstr(  c.reject_solut(res) )
    print(f"runTime:{str(end_run - start_run)[:-7]}, TotTime:{str(end_run-startTime)[:-7]}, State:{sol['state']},")
 
    #-----------------------------
    # Save sol
    state_Sol.append(sol) 
    
    X[['Side','idRule']], Y[['Side','idRule']] = ('X',i),('Y',i)
    data_Solut.append([X,Y])
    i = i+1
    

In [ ]:
# Save state of the tested rules
pd.DataFrame(state_Sol).to_csv('SCE_state.csv')
pd.DataFrame(state_Sol)

In [ ]:
# save the rule with range attribute
pd.concat([pd.concat(d).reset_index().set_index('idRule') for d in data_Solut]).reset_index().to_csv('SCE_solutions.csv',index=False)
pd.concat([pd.concat(d).reset_index().set_index('idRule') for d in data_Solut]).reset_index()